In [ ]:
%matplotlib inline

In [ ]:
#Required to fetch envrio variables 
import os

import pandas as pd
import geopandas
import sqlite3
from shapely import wkt
import folium
from mapboxgl.viz import *
from mapboxgl.utils import *


# Must be a public token, starting with `pk`
token = os.environ['Mapbox_Public_Token']

In [ ]:
#Write out all current areas to a geojson file.
conn = sqlite3.connect('cap_data.db')
query = "select * from cap_poly"
df = pd.read_sql_query(query,conn)
# Convert WKT data in polygon column to shapely format for gdf
df['polygon'] = df['polygon'].apply(wkt.loads)
# Create GeoDataFrame
gdf = geopandas.GeoDataFrame(df, geometry='polygon')
gdf.head()
gdf.to_file("cdn_polys.geojson", driver='GeoJSON')


In [ ]:
conn = sqlite3.connect('cap_data.db')
query = "select cap_alerts.refID, cap_info.infoID, cap_info.headline, cap_info.responseType, cap_alerts.msgType, cap_info.expires,cap_info.event,cap_info.category,cap_info.certainty,cap_info.severity,cap_poly.areaDesc, cap_info.description, cap_poly.polygon from cap_alerts, cap_info, cap_area, cap_poly where cap_alerts.refID = cap_info.refID and cap_info.refid = cap_area.refID and cap_info.infoid = cap_area.infoID and cap_area.areaDesc = cap_poly.areaDesc and cap_alerts.status = 'Actual' and cap_alerts.refID = 'cap-pac@canada.ca,urn:oid:2.49.0.1.124.0774176025.2020,2020-07-14T06:42:44-00:00'"
df1 = pd.read_sql_query(query,conn)
# Convert WKT data in polygon column to shapely format for gdf
df1['polygon'] = df1['polygon'].apply(wkt.loads)
# Create GeoDataFrame
gdf1 = geopandas.GeoDataFrame(df1, geometry='polygon')
# Load geojson file with areas to intersect
fname = "map.geojson"
gdf2 = geopandas.read_file(fname)
# Intersect dataframe results
res_intersection = geopandas.overlay(gdf1, gdf2, how='intersection')
if res_intersection.empty is False:
    # Dissolve data based on ID and info_count 
    textdf = res_intersection.dissolve(by=['refID', 'infoID'], aggfunc='first')
    for index, row in textdf.iterrows():
        print(row['Name'] + ": " + row['headline'] + "\nResponse: " +  row['responseType']  + "\nType: " + row['msgType'] + "\nDescription: " + row['description'])


In [ ]:
conn = sqlite3.connect('cap_data.db')
#query = "select * from cap_alerts"
#query = "select cap_alerts.refID, cap_alerts.identifier, cap_alerts.sender, cap_alerts.sent, cap_alerts.status, cap_alerts.msgType, cap_alerts.rev_refid, cap_info.infoID, cap_info.expires,cap_info.event,cap_info.category,cap_info.certainty,cap_info.severity,cap_info.headline,cap_info.description,cap_info.instruction,cap_poly.polygon from cap_alerts, cap_info, cap_area, cap_poly where cap_alerts.refID = cap_info.refID and cap_info.refid = cap_area.refID and cap_info.infoid = cap_area.infoID and cap_area.areaDesc = cap_poly.areaDesc and datetime(cap_info.expires) >= datetime('now') ORDER BY cap_info.expires ASC"
query = """select cap_alerts.refID,
	cap_alerts.identifier, 
	cap_alerts.sender, 
	cap_alerts.sent, 
	cap_alerts.status,
	cap_alerts.msgType,
	cap_info.infoID,    
	cap_info.expires,
	cap_info.event,
	cap_info.category,
	cap_info.responseType,
	cap_info.certainty,
	cap_info.severity,
	cap_info.urgency,
	cap_info.headline,
	cap_info.description,
	cap_info.instruction,
	cap_poly.polygon,
	cap_alerts.rev_refID
	from cap_alerts, cap_info, cap_area, cap_poly 
	where cap_alerts.refID = cap_info.refID 
	and cap_info.refid = cap_area.refID 
	and cap_info.infoid = cap_area.infoID 
	and cap_area.areaDesc = cap_poly.areaDesc
	and datetime(cap_info.expires) >= datetime('now')
	and rev_refID is NULL
	ORDER BY cap_alerts.sent DESC
"""
df = pd.read_sql_query(query,conn)


In [ ]:
df.head()

In [ ]:
# Convert WKT data in polygon column to shapely format for gdf
df['polygon'] = df['polygon'].apply(wkt.loads)
#df.head()

In [ ]:
# Create GeoDataFrame
gdf = geopandas.GeoDataFrame(df, geometry='polygon')

In [ ]:
# Dissolve data based on ID and info_count 
gdf = gdf.dissolve(by=['identifier', 'infoID'], aggfunc='first')

In [ ]:
import matplotlib.pyplot as plt

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world[world.name == 'Canada'].plot(
    color='white', edgecolor='black')

gdf.plot(ax=ax, color='red')
gdf.plot(color='red')
plt.show()

In [ ]:
gdf.count()

In [ ]:
gdf.drop('sender', axis=1)
gdf.drop('status', axis=1)
gdf.drop('msgType', axis=1)
gdf.drop('event', axis=1)
gdf.drop('category', axis=1)
gdf.drop('rev_refID', axis=1)

# Write out to GeoJson file.
gdf.to_file("cdn_alerts.geojson", driver='GeoJSON')
gdf.head()

In [ ]:
#gdf.to_file("ShapeFiles/cdn_alerts.shp", driver='ESRI Shapefile')

In [ ]:
#import fiona
#fiona.supported_drivers

In [ ]:
match_color_stops = [['AllClear', 'rgb(46,204,113)'],
                     ['Monitor', 'rgb(231,76,60)']]
# https://raw.githubusercontent.com/camwatson/SpatialDataSets/master/Canada_Alerts_English.geojson"
viz = ChoroplethViz("./cdn_alerts.geojson", 
                    access_token=token,
                    color_property='responseType', 
                    color_stops=match_color_stops, 
                    color_function_type='match', 
                    color_default='rgba(52,73,94,0.5)', 
                    opacity=0.8, 
                    center=(-96, 55), 
                    zoom=3, 
                    below_layer='waterway-label')
viz.show()



In [ ]:
m = folium.Map(
    location=[55, -130],
    zoom_start=2  # Limited levels of zoom for free Mapbox tiles.
)

folium.GeoJson(
    gdf.to_json(),
    name='geojson',
).add_to(m)


In [ ]:
m